In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import colour as color
import numpy as np
import cv2
from colour_checker_detection import detect_colour_checkers_segmentation as detect_color_chart


def sharpen_image(image, kernel_size=3, sigma=1.0, alpha=1.5, beta=-0.5, gamma=0):
    """
    Sharpen image using Gaussian blur for edge detection and add it back to the original image.
    
    Parameters:
    - image: Input image
    - kernel_size: Size of the Gaussian kernel. Larger kernel_size implies more blurring.
    - sigma: Standard deviation of the Gaussian kernel.
    - alpha: Scale factor for the original image.
    - beta: Scale factor for the sharpening effect.
    - gamma: Scalar added to each sum (optional).
    
    Returns:
    - sharpened_image: The sharpened image.
    """
    # Ensure kernel_size is odd
    if kernel_size % 2 == 0:
        kernel_size += 1
    
    # Use Gaussian blur to get the "unsharp" part of the image
    blurred = cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)
    
    # Calculate the sharpened image by adding the scaled difference (details) back to the original
    sharpened_image = cv2.addWeighted(image, alpha, blurred, beta, gamma)
    
    return sharpened_image


frame_path = '/home/patrick/Videos/colorchecks/rwg-log3g10-ffmpeg-yuyv.png'

color_loaded = color.read_image(frame_path)
chart_subset = color_loaded[400:800, 0:600, :]
# Scale down the chart by 3x
chart_subset = cv2.resize(chart_subset, (0, 0), fx=0.3, fy=0.3)
color_chart = detect_color_chart(chart_subset, additional_data=True, show=True)

# Increase contrast of chart_subset with cv2
chart_subset_2 = cv2.convertScaleAbs(chart_subset * 255, alpha=1.5, beta=0)
chart_subset_2 = sharpen_image(chart_subset_2, kernel_size=10, sigma=1.0, alpha=1.5, beta=-0.5, gamma=0)
# sharpen again
sharpen_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
chart_subset_2 = cv2.filter2D(chart_subset_2, -1, sharpen_kernel)
chart_subset_2 = cv2.filter2D(chart_subset_2, -1, sharpen_kernel)
color_chart_2 = detect_color_chart(chart_subset_2 / 255, additional_data=True, show=True)


# Plot the color chart and the source image
plt.figure(1, figsize=(20, 10))
plt.imshow(color_loaded)
plt.figure(1, figsize=(20, 10))
plt.imshow(chart_subset)
plt.figure(1, figsize=(20, 10))
plt.imshow(chart_subset_2)
print(color_chart, color_chart_2)